In [7]:
import SimpleITK as sitk
import numpy as np
from loguru import logger
import os,sys

from cardiac import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
img = sitk.ReadImage("../../TempCardiacData/Case_03AD/Images/Case_03AD.nii.gz")

In [42]:
settings = {
    'outputFormat'          : 'Auto_{0}.nii.gz',
    'atlasSettings'         : {
                                'atlasIdList':               ['04D1','1FA5','435F','8505'],
                                'atlasStructures':           ['COR','LAD']
                              },
    'lungMaskSettings'      : {
                                'coronalExpansion':          15,
                                'axialExpansion':            5,
                                'sagittalExpansion':         0,
                                'lowerNormalisedThreshold':  -0.1,
                                'upperNormalisedThreshold':  0.4,
                                'voxelCountThreshold':       5e4
                              },
    'rigidSettings'         : {


                              },
    'deformableSettings'    : {
                                'resolutionStaging':        [8,4,2],
                                'iterationStaging':         [5,5,5],
                                'ncores':4
                              },
    'IARSettings'    : {
                                'referenceStructure':       'COR',
                                'smoothDistanceMaps':       True,
                                'smoothSigma':              1,
                                'zScoreStatistic':          'MAD',
                                'outlierMethod':            'IQR',
                                'outlierFactor':            1.5,
                                'minBestAtlases':           4
                              },
    'labelFusionSettings'   : {
                                'voteType':                 'local'
                              }
}

In [26]:
"""
Initialisation - Read in atlases
- image files
- structure files

    Atlas structure:
    'ID': 'Original': 'CT Image'    : sitk.Image
                      'Struct A'    : sitk.Image
                      'Struct B'    : sitk.Image
          'RIR'     : 'CT Image'    : sitk.Image
                      'Transform'   : transform parameter map
                      'Struct A'    : sitk.Image
                      'Struct B'    : sitk.Image
          'DIR'     : 'CT Image'    : sitk.Image
                      'Transform'   : displacement field transform
                      'Weight Map'  : sitk.Image
                      'Struct A'    : sitk.Image
                      'Struct B'    : sitk.Image


"""
logger.info('')
# Settings
atlasIdList = settings['atlasSettings']['atlasIdList']
atlasStructures = settings['atlasSettings']['atlasStructures']

atlasSet = {}
for atlasId in atlasIdList:
    atlasSet[atlasId] = {}
    atlasSet[atlasId]['Original'] = {}

    atlasSet[atlasId]['Original']['CT Image'] = sitk.ReadImage('../../TempCardiacData/Case_{0}/Images/Case_{0}.nii.gz'.format(atlasId))

    for struct in atlasStructures:
        atlasSet[atlasId]['Original'][struct] = sitk.ReadImage('../../TempCardiacData/Case_{0}/Structures/Case_{0}_{1}.nii.gz'.format(atlasId, struct))


2019-08-14 13:00:31.468 | INFO     | __main__:<module>:22 - 


In [27]:
"""
Step 1 - Automatic cropping using the lung volume
- Airways are segmented
- A bounding box is defined
- Potential expansion of the bounding box to ensure entire heart volume is enclosed
- Target image is cropped
"""
# Settings
sagittalExpansion = settings['lungMaskSettings']['sagittalExpansion']
coronalExpansion  = settings['lungMaskSettings']['coronalExpansion']
axialExpansion    = settings['lungMaskSettings']['axialExpansion']

lowerNormalisedThreshold = settings['lungMaskSettings']['lowerNormalisedThreshold']
upperNormalisedThreshold = settings['lungMaskSettings']['upperNormalisedThreshold']
voxelCountThreshold      = settings['lungMaskSettings']['voxelCountThreshold']

# Get the bounding box containing the lungs
lungBoundingBox, _ = AutoLungSegment(img, l=lowerNormalisedThreshold, u=upperNormalisedThreshold, NPthresh=voxelCountThreshold)

# Add an optional expansion
sag0 = max([lungBoundingBox[0] - sagittalExpansion, 0])
cor0 = max([lungBoundingBox[1] - coronalExpansion,  0])
ax0  = max([lungBoundingBox[2] - axialExpansion,    0])

sagD = min([lungBoundingBox[3] + sagittalExpansion, img.GetSize()[0] - sag0])
corD = min([lungBoundingBox[4] + coronalExpansion,  img.GetSize()[1] - cor0])
axD  = min([lungBoundingBox[5] + axialExpansion,    img.GetSize()[2] - ax0])

cropBox = (sag0, cor0, ax0, sagD, corD, axD)

# Crop the image down
imgCrop = CropImage(img, cropBox)

In [28]:
"""
Step 2 - Rigid registration of target images
- Individual atlas images are registered to the target
- The transformation is used to propagate the labels onto the target
"""
for atlasId in atlasIdList:
    # Register the atlases
    atlasSet[atlasId]['RIR'] = {}
    atlasImage = atlasSet[atlasId]['Original']['CT Image']
    rigidImage, rigidTfm = RigidRegistration(imgCrop, atlasImage,'RigidTransformParameters.txt')

    # Save in the atlas dict
    atlasSet[atlasId]['RIR']['CT Image']  = rigidImage
    atlasSet[atlasId]['RIR']['Transform'] = rigidTfm


    for struct in atlasStructures:
        inputStruct = atlasSet[atlasId]['Original'][struct]
        atlasSet[atlasId]['RIR'][struct] = RigidPropagation(imgCrop, inputStruct, rigidTfm, structure=True)


In [29]:
"""
Step 3 - Deformable image registration
- Using Fast Symmetric Diffeomorphic Demons
"""
# Settings
resolutionStaging = settings['deformableSettings']['resolutionStaging']
iterationStaging  = settings['deformableSettings']['iterationStaging']
ncores            = settings['deformableSettings']['ncores']

for atlasId in atlasIdList:
    # Register the atlases
    atlasSet[atlasId]['DIR'] = {}
    atlasImage = atlasSet[atlasId]['RIR']['CT Image']
    deformImage, deformField = FastSymmetricForcesDemonsRegistration(imgCrop, atlasImage, resolutionStaging=resolutionStaging, iterationStaging=iterationStaging, ncores=ncores)

    # Save in the atlas dict
    atlasSet[atlasId]['DIR']['CT Image']  = deformImage
    atlasSet[atlasId]['DIR']['Transform'] = deformField

    for struct in atlasStructures:
        inputStruct = atlasSet[atlasId]['RIR'][struct]
        atlasSet[atlasId]['DIR'][struct] = ApplyField(inputStruct, deformField, 1, 0)

  1 = 64209.13822
  2 = 53855.53241
  3 = 45580.64636
  4 = 40837.58322
  5 = 38614.79373
  1 = 45851.04551
  2 = 40849.63444
  3 = 38461.55976
  4 = 37011.06491
  5 = 35988.55789
  1 = 41639.26014
  2 = 38488.26768
  3 = 36742.65808
  4 = 35622.09042
  5 = 34819.46381
  1 = 54836.99912
  2 = 40905.58669
  3 = 30410.08962
  4 = 26424.06079
  5 = 23792.32542
  1 = 29321.99930
  2 = 23749.33324
  3 = 20966.94922
  4 = 19254.92116
  5 = 18035.54312
  1 = 27343.45096
  2 = 23927.82083
  3 = 22140.57232
  4 = 21025.63337
  5 = 20240.20752
  1 = 82674.91056
  2 = 71336.54844
  3 = 63353.13186
  4 = 58460.56413
  5 = 55007.15249
  1 = 71390.11532
  2 = 66005.55144
  3 = 62747.52031
  4 = 60437.05685
  5 = 58608.37371
  1 = 71198.26810
  2 = 67597.98325
  3 = 65437.00068
  4 = 63929.95891
  5 = 62783.19846
  1 = 125671.99466
  2 = 110305.16604
  3 = 96296.45845
  4 = 85871.79051
  5 = 78560.16995
  1 = 94523.17497
  2 = 87582.16388
  3 = 82987.29311
  4 = 79511.15873
  5 = 76655.55908
  1 = 90

In [57]:
"""
Step 4 - Iterative atlas removal
- This is an automatic process that will attempt to remove inconsistent atlases from the entire set

"""

# Compute weight maps
for atlasId in atlasIdList:
    atlasImage = atlasSet[atlasId]['DIR']['CT Image']
    weightMap = computeWeightMap(imgCrop, atlasImage)
    atlasSet[atlasId]['DIR']['Weight Map'] = weightMap
    
    
combinedLabelDict = combineLabels(atlasSet, 'COR')

referenceStructure  = settings['IARSettings']['referenceStructure']
smoothDistanceMaps  = settings['IARSettings']['smoothDistanceMaps']
smoothSigma         = settings['IARSettings']['smoothSigma']
zScoreStatistic     = settings['IARSettings']['zScoreStatistic']
outlierMethod       = settings['IARSettings']['outlierMethod']
outlierFactor       = settings['IARSettings']['outlierFactor']
minBestAtlases      = settings['IARSettings']['minBestAtlases']

atlasSet = IAR(atlasSet         = atlasSet,
               structureName    = referenceStructure,
               smoothMaps       = smoothDistanceMaps,
               smoothSigma      = smoothSigma,
               zScore           = zScoreStatistic,
               outlierMethod    = outlierMethod,
               minBestAtlases   = minBestAtlases,
               N_factor         = outlierFactor,
               logFile          = 'IAR_{0}.log'.format(datetime.datetime.now()),
               debug            = False,
               iteration        = 0,
               singleStep       = False)

Iterative atlas removal: 
  Beginning process
  Less than 12 atlases, resolution set: 3x3 sqr deg
  Calculating surface distance maps: 
    04D1     1FA5     435F     8505 
    MAD zero count: 489
    MAD zero count: 508
    MAD zero count: 401
    MAD zero count: 300
  Analysing results
   Outlier limit: 18.855
      04D1: Q = 15.904 [KEEP]
      1FA5: Q = 07.544 [KEEP]
      435F: Q = 10.187 [KEEP]
      8505: Q = 04.530 [KEEP]
  End point reached. Keeping:
   ['04D1', '1FA5', '435F', '8505']


In [58]:
"""
Step 5 - Label Fusion
"""
combinedLabelDict = combineLabels(atlasSet, atlasStructures)

In [62]:
"""
Step 6 - Paste the cropped structure into the original image space
"""

outputFormat = settings['outputFormat']

templateIm = sitk.Cast((img * 0),sitk.sitkUInt8)

for structureName in atlasStructures:
    binaryStruct = processProbabilityImage(combinedLabelDict[structureName], 0.5)
    pasteImg = sitk.Paste(templateIm, binaryStruct, binaryStruct.GetSize(), (0,0,0), (sag0, cor0, ax0))

    # Write the mask to a file in the working_dir
    mask_file = os.path.join(
        '.', outputFormat.format(structureName))
    sitk.WriteImage(pasteImg, mask_file)


In [90]:
import vtk
from vtk.util.numpy_support import numpy_to_vtk

In [66]:
reader = vtk.vtkNIFTIImageReader()
reader.SetFileName('./Auto_COR.nii.gz')
reader.Update()
outputImage = reader.GetOutput()
outputImage

(vtkCommonDataModelPython.vtkImageData)0x7fcb173e99a8

In [86]:
test = sitk.ReadImage('./Auto_COR.nii.gz')
test_arr = sitk.GetArrayFromImage(test)

dataImporter = vtk.vtkImageImport()
# The preaviusly created array is converted to a string of chars and imported.
data_string = test_arr#.tostring()
dataImporter.CopyImportVoidPointer(data_string, len(data_string))
# The type of the newly imported data is set to unsigned char (uint8)
dataImporter.SetDataScalarTypeToUnsignedChar()
# Because the data that is imported only contains an intensity value (it isnt RGB-coded or someting similar), the importer
# must be told this is the case.
dataImporter.SetNumberOfScalarComponents(1)

outputImage_sitk = dataImporter.GetOutput()

In [94]:
flat_data_array = test_arr.transpose(2,1,0).flatten();
        
vtk_data_array = numpy_to_vtk(flat_data_array)
image = vtk.vtkImageData() 
points = image.GetPointData()
points.SetScalars(vtk_data_array)


image.SetDimensions(test_arr.shape)
#image.Update()

AttributeError: 'vtkCommonDataModelPython.vtkImageData' object has no attribute 'Update'

In [87]:
outputImage_sitk.GetExtent(), outputImage.GetNumberOfPoints(), outputImage.GetPointData().GetScalars()

((0, -1, 0, -1, 0, -1),
 31195136,
 (vtkCommonCorePython.vtkUnsignedCharArray)0x7fcb15f1e288)

In [88]:
outputImage_sitk.GetExtent(), outputImage_sitk.GetNumberOfPoints(),

((0, -1, 0, -1, 0, -1), 0)